In [6]:
#Prerequisite is to run CLI loging to the target Suscbription
#az login --tenant 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [7]:
def get_dates():
    date_start = input("Starting date - Format of request (EST date): '2023-01-07 09:00')")
    date_end = input("Ending date - Format of request (EST date): '2023-01-07 19:00')")
    #return (date_start, date_end)
    return (str(date_start), str(date_end))

In [8]:
def get_dataset(environment, date_start, date_end):
    import os
    import pandas as pd
    from datetime import datetime, timezone
    from azure.monitor.query import LogsQueryClient, LogsQueryStatus
    from azure.identity import DefaultAzureCredential
    from azure.core.exceptions import HttpResponseError
    import datetime
    from datetime import timezone, timedelta
    from dotenv import load_dotenv
    import pytz
    import imp

    #Get Microsoft Azure environement variables (see pre-requisites to create the environement variables)
    env = imp.load_source('env','env.txt')
    environment = environment

    #Choose the environment LAB or NP
    if (environment == 'LAB'):
        azure_log_customer_id = env.vzpoc_AZURE_WORKSPACE_ID
    else:
        azure_log_customer_id = env.vzpmec_AZURE_WORKSPACE_ID

    #credential = DefaultAzureCredential()
    credential = DefaultAzureCredential(AllowMultiTenantAuthentication = True)
    client = LogsQueryClient(credential)
    
    date_start=datetime.datetime.strptime(date_start, '%Y-%m-%d %H:%M')
    date_end=datetime.datetime.strptime(date_end, '%Y-%m-%d %H:%M')

    #Convert from EST to UTC time
    date_start=date_start + timedelta(hours=5)
    date_end=date_end + timedelta(hours=5)

    #Set time zone to EST
    ESTTimeDelta = datetime.timedelta(hours=-5)
    ESTTZObject = datetime.timezone(ESTTimeDelta,name="EST")

    #Set required query
    query_Sessions = "MicrosoftAzureBastionAuditLogs| project ClientIpAddress, Duration, Protocol, SessionStartTime, SessionEndTime, OperationName, UserName, Message,TargetVMIPAddress, _ResourceId"
    query = query_Sessions

    try:
        response = client.query_workspace(
            workspace_id=azure_log_customer_id,
            query=query,
            timespan=(date_start, date_end)
            )
        if response.status == LogsQueryStatus.PARTIAL:
            error = response.partial_error
            data = response.partial_data
            print(error.message)
        elif response.status == LogsQueryStatus.SUCCESS:
            data = response.tables
        for table in data:
            df = pd.DataFrame(data=table.rows, columns=table.columns)
            #print(df)

        #https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
        pd.options.mode.chained_assignment = None

        if (df[df['Message'] == 'Successfully Disconnected'].count()[0] > 0):
            #print (df_data_Sessions.shape[0])

            #Add Nan where string is empty
            df=df.mask(df == '')

            #Convert string time to datetime
            df['SessionEndTime']=pd.to_datetime(df['SessionEndTime'], format='%m/%d/%Y %I:%M:%S %p')
            df['SessionStartTime']=pd.to_datetime(df['SessionStartTime'], format='%Y-%m-%dT%H:%M:%S')

            #Set Timezone
            df['SessionEndTime']=df['SessionEndTime'].dt.tz_localize(timezone.utc)
            df['SessionEndTime']=df['SessionEndTime'].dt.tz_convert(ESTTZObject)
            df['SessionStartTime']=df['SessionStartTime'].dt.tz_convert(ESTTZObject)

            #Extract AzureBastion
            for ind in df.index:
                df['_ResourceId'][ind]=df['_ResourceId'][ind].split('/')[-1]

        else:
            #print (df_sessions.shape[0])
            print ("There is no identified session in the chosen time range")
        
        return df
            
    except HttpResponseError as err:
        print("something fatal happened")
        print (err)

In [9]:
 def graph_dataset_session(environement):
    import pandas as pd
    import pytz
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    import matplotlib.ticker as plticker
    import datetime
    from datetime import timezone, timedelta

    #Remove the warning for chained changes
    #https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
    pd.options.mode.chained_assignment = None

    #Setenvironment
    environement = environement
    
    #Get Starting and ending dates
    date_start, date_end = get_dates()
    d1=date_start
    d2=date_end
    df=get_dataset(environement, date_start, date_end)

    #print (df.shape[0])
    if (df.shape[0] > 0):
    #if (df[df['Message'] == 'Successfully Disconnected'].count()[0] > 0):
        #print (df_data_Sessions.shape[0])

        #Slice data to new DataFrame
        df_sessions=df[['UserName','SessionStartTime', 'SessionEndTime', '_ResourceId', 'TargetVMIPAddress']][(df['Message']=='Successfully Disconnected')].sort_values(by='SessionStartTime', ascending=True).reset_index()

        # Plot the data in to a graph
        fig, ax = plt.subplots(figsize=(20,12))
        #import matplotlib.dates as mdates

        for ind in df_sessions.index:
            x = [df_sessions['SessionStartTime'][ind], df_sessions['SessionEndTime'][ind]]
            y = [ind+1, ind+1]
            ax.plot(x, y, linestyle='-', marker='')
            ax.text(x[0], y[0]+0.1, 'Session {} from {}\n using {} to connect to {}'.format(ind+1, df_sessions['UserName'][ind], df_sessions['_ResourceId'][ind], df_sessions['TargetVMIPAddress'][ind]), fontsize = 5)

        # Define the date format & Timezone
        my_tz = pytz.timezone('EST')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d-%H:%M', tz=my_tz))            

        #x axis configuration
        ax.xaxis_date()
        hours = mdates.HourLocator(interval = 1)  #
        ax.xaxis.set_major_locator(hours)
        ax.xaxis.grid(linestyle='--', linewidth=0.5)
        ax.tick_params(axis='both', which='major', labelsize=7)
        ax.tick_params(axis='both', which='minor', labelsize=7)
        ax.set_xlim([df_sessions['SessionStartTime'].min()- timedelta(hours=0.5), df_sessions['SessionEndTime'].max() + timedelta(hours=5)])
        ax.set_xlabel('Sessions time (EST) & duration', fontsize=10)
        fig.autofmt_xdate() 

        #y axis configuration
        ax.set_ylim([0, df_sessions.shape[0] + 1])
        loc = plticker.MultipleLocator(base=1.0) # this locator puts ticks at regular intervals
        ax.yaxis.set_major_locator(loc)
        ax.set_ylabel('Sessions', fontsize='medium') 

        #Set Graph Title
        ax.set_title('{} Bastion Sessions from {} to {} EST'.format(df_sessions.shape[0], d1, d2))
    else:
        #print (df_sessions.shape[0])
        print ("There is no identified session in the chosen time range")

In [11]:
graph_dataset_session('NP')